In [6]:
# pip install requests
# pip install google_images_download
# pip install beautifulsoup4

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
URL = 'https://en.wikipedia.org/wiki/List_of_Indian_artists'
page = requests.get(URL)
page

<Response [200]>

In [4]:
soup=BeautifulSoup(page.content, 'html.parser')
soup


<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of Indian artists - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"dad732c1-ea4a-48da-8cf2-05708387fbe1","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_Indian_artists","wgTitle":"List of Indian artists","wgCurRevisionId":963183907,"wgRevisionId":963183907,"wgArticleId":4457757,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Lists of Indian artists"],"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgRelevantPageName":"List_of_Indian_artists","wgRele

In [5]:
alllinks=soup.findAll('a')
len(alllinks)

261

In [6]:
URLsOfAllArtists=[]
for i in alllinks:
    try:
        URLsOfAllArtists.append(URL[:-len('/wiki/List_of_Indian_artists')]+i['href'])
    except KeyError:
        pass
    
URLsOfAllArtists=URLsOfAllArtists[4:]

In [30]:
skip=0
for i in range(len(URLsOfAllArtists)):
    if len(URLsOfAllArtists[i])>61 or (URLsOfAllArtists[i].find("Indian_people")>-1):
        skip+=1
        #print(URLsOfAllArtists[i])
        URLsOfAllArtists=URLsOfAllArtists[:i]+URLsOfAllArtists[i+1:]
    if skip>2:
        URLsOfAllArtists=URLsOfAllArtists[:i]
        break
    
URLsOfAllArtists=URLsOfAllArtists[:-2]
len(URLsOfAllArtists)

94

In [16]:
#Remove tags, logic skip everything from '<' to '>'
def remtag(a):   
    b=''
    f=0
    for i in a:
        if i=='<':
            f=1
        if f==0:
            b=b+i
        if i=='>':
            f=0
    return b
    
def remsqb(a):
    b=''
    f=0
    for i in a:
        if i=='[':
            f=1
        if f==0:
            b=b+i
        if i==']':
            f=0
    return b
    
def remb(a):
    return remsqb(remtag(a))

#check if character is number
def isnumber(a):
    try:
        int(a)
        return True
    except ValueError:
        return False
    
#Extract date from text
def extdate(a):
    y=0
    year=''
    for i in range(len(a)):
        if y==4:
            year=a[i-4:i]
            break
#check if character is number
        if isnumber(a[i]):
            y+=1
        else:
            y=0
           
    return year

#extract name from text
def extname(a):
    b=''
    spacelist=[]
    for i in range(len(a)):
        if i>25:
            b=a[:spacelist[1]]
            if b[-3:-1]=='is':
                b=a[:spacelist[0]]
            break
        if a[i]==' ':
            spacelist.append(i)
        if a[i]=='(':
            break
        b+=a[i]
    return b

def repplus(a):
    for i in range(len(a)):
        if a[i]==' ':
            a[i]='+'
            
    return a

In [27]:
jsondict={
    "title": 
    {
        "media": {
          "url": "https://learnodo-newtonic.com/wp-content/uploads/2019/05/Famous-Indian-Artists-Featured.webp",
          "caption": "Famous Indian Artists",
          "credit": "learnodo-newtonic.com/<a href='https://learnodo-newtonic.com/wp-content/uploads/2019/05/Famous-Indian-Artists-Featured.webp'>picture</a>"
        },
        "text": {
          "headline": "Indian Artists<br/> 1848 - 2015",
          "text": "<p>This is a timeline of some notable artists who were born in India or have a strong association with India. </p>"
        }
    },
    "events": [
        
    ]
}

In [28]:

for i in URLsOfAllArtists:
    
    AURL=requests.get(i)
    AURL=BeautifulSoup(AURL.content,'html.parser')
    Ptags=AURL.findAll('p')
    text=''
    for i in Ptags[:4]:
        text+=remb(str(i))
        if len(text)>400:
            break

    date=extdate(text)
    headline=extname(text)
    

    jsondict2={"media":{"url":"","caption":"","credit":""},"start_date":{"year":str(date)},"text":{"headline":str(headline),"text":str(text)}}
    jsondict["events"].append(jsondict2)

import json
with open("artistsdata.json", "w") as outfile:  
    json.dump(jsondict, outfile) 

In [29]:
jsondict

{'title': {'media': {'url': 'https://learnodo-newtonic.com/wp-content/uploads/2019/05/Famous-Indian-Artists-Featured.webp',
   'caption': 'Famous Indian Artists',
   'credit': "learnodo-newtonic.com/<a href='https://learnodo-newtonic.com/wp-content/uploads/2019/05/Famous-Indian-Artists-Featured.webp'>picture</a>"},
  'text': {'headline': 'Indian Artists<br/> 1848 - 2015',
   'text': '<p>This is a timeline of some notable artists who were born in India or have a strong association with India. </p>'}},
 'events': [{'media': {'url': '', 'caption': '', 'credit': ''},
   'start_date': {'year': '1913'},
   'text': {'headline': '\nAmrita Sher-Gil ',
    'text': '\nAmrita Sher-Gil (30 January 1913 – 5 December 1941) was a Hungarian-Indian painter. She has been called "one of the greatest avant-garde women artists of the early 20th century" and a "pioneer" in modern Indian art. Drawn to painting from an early age, Sher-Gil started getting formal lessons in the art, at the age of eight. She firs

In [19]:
ImURL="https://en.wikipedia.org/wiki/Kalipada_Ghoshal"
#URL = 'https://en.wikipedia.org/wiki/List_ofian_artists'
page = requests.get(ImURL)
page

<Response [200]>

In [ ]:
soup=BeautifulSoup(page.content, 'html.parser')
soup.prettify()

In [ ]:

soup2=soup.find_all('table')
soup2[1].find_all('a')

In [ ]:
req

In [ ]:
from google_images_download import google_images_download
response = google_images_download.googleimagesdownload()  

In [ ]:
headline2="Amrita sher-Gil"
search_queries=["Indian Artist "+ headline2]

  
def downloadimages(query): 
    # keywords is the search query 
    # format is the image file format 
    # limit is the number of images to be downloaded 
    # print urs is to print the image file url 
    # size is the image size which can 
    # be specified manually ("large, medium, icon") 
    # aspect ratio denotes the height width ratio 
    # of images to download. ("tall, square, wide, panoramic") 
    arguments = {"keywords": query, 
                 #"format": "jpg", 
                 "limit":2, 
                 "print_urls":True, 
                 #"size": "medium", 
                 } 
    a=response.download(arguments)
    print(a)
#     try: 
#         response.download(arguments) 
      
#     # Handling File NotFound Error     
#     except FileNotFoundError:  
#         arguments = {"keywords": query, 
#                      "format": "jpg", 
#                      "limit":1, 
#                      "print_urls":True,  
#                      "size": "medium"} 
                       
#         # Providing arguments for the searched query 
#         try: 
#             # Downloading the photos based 
#             # on the given arguments 
#             response.download(arguments)  
#         except: 
#             pass

for query in search_queries: 
    print(downloadimages(query)  )
    print()  

In [ ]:
from google_images_download import google_images_download   #importing the library

response = google_images_download.googleimagesdownload()   #class instantiation

arguments = {"keywords":"Polar bears,baloons,Beaches","limit":20,"print_urls":True}   #creating list of arguments
paths = response.download(arguments)   #passing the arguments to the function
print(paths)   #printing absolute paths of the downloaded images

In [ ]:
temp=requests.get('https://en.wikipedia.org/wiki/Raja_Ravi_Varma')
temp=BeautifulSoup(temp.content,'html.parser')

In [ ]:
temp

In [ ]:
Ptemp=temp.findAll('p')

In [ ]:
Ptemp

In [ ]:
Ptemp[1]

In [ ]:
def remtag(a):
    b=''
    f=0
    for i in a:
        if i=='<':
            f=1
        if f==0:
            b=b+i
        if i=='>':
            f=0
    return b
    
def remsqb(a):
    b=''
    f=0
    for i in a:
        if i=='[':
            f=1
        if f==0:
            b=b+i
        if i==']':
            f=0
    return b
    
text=remsqb(remtag(str(Ptemp[0])+str(Ptemp[1])))
print(text)


In [ ]:
import json

In [ ]:
def isnumber(a):
    try:
        int(a)
        return True
    except ValueError:
        return False

def extdate(a):
    y=0
    year=''
    for i in range(len(a)):
        if y==4:
            year=a[i-4:i]
            break
        if isnumber(a[i]):
            y+=1
        else:
            y=0
           
    return year
print(extdate(text))

In [ ]:
#extract date
#take 1 to 3 para and limit number of characters. cut off to last sentence.

In [ ]:
def extname(a):
    b=''
    for i in a:
        if i=='(':
            break
        b+=i
    return b

##problems - ( missing, need first two words
##date missing - remove or use diff source or put in end as other notable artists


In [ ]:
"422 Amrita Sher-Gil (30 January 1913 – 5 December 1941) was a Hungarian-Indian painter. She has been called one 


In [ ]:
def remtag(a):
    b=''
    f=0
    for i in a:
        
        if i=='<':
            f=1
        if f==0:
            b=b+i
        if i=='>':
            f=0
        
    print(b)
remtag(str("<p><b>Amrita Sher-Gil</b> (30 January 1913 – 5 December 1941)"))

def isnumber(a):
    try:
        int(a)
        return True
    except ValueError:
        return False

In [ ]:
# print("<p><b>Amrita Sher-Gil</b> (30 January 1913 – 5 December 1941)")
# def removetag(a):
    
#     l=len(a)
#     i=0
#     while i<l:
#         if a[i] == '<':
#             a=a[:i]+a[i+1:]
#             l-=1
#             #i-=1
#             while a[i] != '>' :
#                 a=a[:i]+a[i+1:]
#                 l-=1
#                 #i-=1
# #         if i<len(a):
# #             print(a[i],end='')
#     print(a)

# #     for i in range(len(a)):
# #         if a[i] == '<':
# #             a=a[:i]+a[i+1:]
# #             while a[i] != '>' :
# #                 a=a[:i]+a[i+1:]
        
# #         if i<len(a):
# #             print(a[i],end='')
            


# removetag(str("<p><b>Amrita Sher-Gil</b> (30 January 1913 – 5 December 1941)"))

# def remab(a):
#     l=len(a)
#     i=0
#     while i<l:
#         if a[i] == '>':
#             print("asd")
#             a=a[:i]+a[i+1:]
#             l-=1
#             i-=1
#         i+=1
#     print(a)
#     return a
# remab(">>Amrita Sher-Gil> (30 January 1913 – 5 December 1941)")


# def remab(a):
#     for i in range(len(a)-1):
#         if a[i] == '>':
#             #print("asd")
#             a=a[:i]+a[i+1:]
#     return a

# remab(">>Amrita Sher-Gil> (30 January 1913 – 5 December 1941)")


# print("<p><b>Amrita Sher-Gil</b> (30 January 1913 – 5 December 1941)")
# def removetag(a):
#     b=''
#     for i in range(len(a)):
#         if a[i] == '<' and i<len(a)-1:
#             a=a[:i]+a[i+1:]
#             while a[i] != '>' :
#                 a=a[:i]+a[i+1:]
#             #a=a[:i]+a[i+1:]
        
# removetag(str("<p><b>Amrita Sher-Gil</b> (30 January 1913 – 5 December 1941)"))